In [33]:
import pandas as pd
import sys
import torch
from torch.utils.data import DataLoader

# Add your system path here
sys_path = r'C:\Users\Max Tost\Desktop\Notebooks\PowerPrediction\ml-project-2-powerpredictors'
sys.path.append(sys_path)

from helpers import *

### Loading the data in a dataframe

In [34]:
data = pd.read_csv(r'C:\Users\Max Tost\Desktop\Notebooks\PowerPrediction\ml-project-2-powerpredictors\data\Training_dataset.csv')
print(data[:3])
look_ahead = 5 # Timesteps that the LSTM will be able to look into the future
data_tensor = shift_dataframe_column(data, 'power_consumption', look_ahead)
print(data_tensor[0])

   year     month  day      hour  power_consumption       ghi      temp  \
0   0.0  0.272727  0.4  0.434783           0.218368  2.318664  0.448423   
1   0.0  0.272727  0.4  0.478261          -0.111426  2.091075  0.567245   
2   0.0  0.272727  0.4  0.521739          -0.166392  1.985427  0.692455   

       wind  is_weekend  is_holiday  is_daylight  
0 -0.837392           0           0            1  
1 -0.368104           0           0            1  
2  0.114987           0           0            1  
   year     month  day      hour  power_consumption       ghi      temp  \
3   0.0  0.272727  0.4  0.565217                NaN  1.270273  0.766559   
4   0.0  0.272727  0.4  0.608696                NaN  1.713181  0.799778   
5   0.0  0.272727  0.4  0.652174           0.218368  1.355604  0.802333   
6   0.0  0.272727  0.4  0.695652          -0.111426  0.664831  0.789557   

       wind  is_weekend  is_holiday  is_daylight  
3  0.018369           0           0            1  
4 -0.174867      

## Creating features and targets with Dataloader

In [35]:
# Creating datasets
train_dataset = MultiTimeSeriesDataset(data_tensor[:int(0.8*len(data_tensor))]) # Using first 80% for training
val_dataset = MultiTimeSeriesDataset(data_tensor[int(0.8*len(data_tensor)):]) # Using last 20% for evaluation 

# DataLoader for batching
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)

In [36]:
for x, y in train_loader:
    print(x.shape, y.shape)
    break

torch.Size([16, 72, 11]) torch.Size([16, 1])


C:\Users\Max Tost\Desktop\Notebooks\PowerPrediction\ml-project-2-powerpredictors\helpers.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32).unsqueeze(0)


## Training Loop

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_prob):
        """
        Initialize the LSTM-based regression model.

        Args:
            input_size (int): Number of input features (e.g., temperature, GHI, etc.).
            hidden_size (int): Number of units in each LSTM layer.
            num_layers (int): Number of stacked LSTM layers.
            output_size (int): Number of output features (e.g., predicted demand, 1 for regression).
            dropout_prob (float): Dropout probability to apply between LSTM layers and before the fully connected layer.
        """
        super(LSTMModel, self).__init__()

        # LSTM Layer
        # - Processes sequential data and learns temporal dependencies.
        # - Supports multiple layers (num_layers) and applies dropout between layers.
        self.lstm = nn.LSTM(
            input_size=input_size, 
            hidden_size=hidden_size, 
            num_layers=num_layers,
            dropout=dropout_prob,
            batch_first=True,
        )

        # Fully Connected (Linear) Layer
        # - Maps the LSTM's hidden state output to the desired output size.
        self.fc = nn.Linear(hidden_size, output_size)

        # Dropout Layer
        # - Reduces overfitting by randomly zeroing some activations during training.
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self, x):
        """
        Forward pass for the LSTM model.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, seq_length, input_size).

        Returns:
            torch.Tensor: Output predictions of shape (batch_size, output_size).
        """
        # LSTM Layer
        # - Returns the full sequence of hidden states and the final hidden/cell state tuple.
        # - We ignore the hidden/cell state tuple here (h_n, c_n).
        _ , (h_f, _) = self.lstm(x)

        # Dropout Layer
        # - Only uses the hidden state from the last time step for prediction.
        # - Applies dropout to prevent overfitting.
        out = self.dropout(h_f[0])  # Shape: (num_layers, batch_size, hidden_size)

        out = 

        # Fully Connected Layer
        # - Maps the LSTM's output to the desired output size (e.g., single regression output).
        out = self.fc(out)  # Shape: (batch_size, output_size)

        return out

In [8]:
# 5 min per epoch here on this cpu, validation loss afer 1 epoch: 154743.1982 (Whatever that means)
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR

# Number of epochs
num_epochs = 5

# Initialize model, loss function, and optimizer
model = LSTMModel(input_size=len(data_tensor[0]), hidden_size=64, num_layers=2, output_size=1, dropout_prob=0.2)
criterion = torch.nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)

# Initialize the cosine annealing scheduler
scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=0.0001)

# A single validation step before training
model.eval()
val_loss = 0.0
with torch.no_grad():
    for x_val, y_val in val_loader:
        val_output = model(x_val)
        val_loss += criterion(val_output, y_val).item()
val_loss /= len(val_loader)
print(f"Epoch 0, Validation Loss: {val_loss:.4f}")

for epoch in range(num_epochs):
    model.train()
    for x, y in train_loader:
        optimizer.zero_grad()

        # Get output
        output = model(x)

        # Compute loss
        loss = criterion(output, y)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

    # Step the scheduler at the end of the epoch
    scheduler.step()

    # Validation (optional)
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for x_val, y_val in val_loader:
            val_output = model(x_val)
            val_loss += criterion(val_output, y_val).item()
    val_loss /= len(val_loader)
    print(f"Epoch {epoch + 1}, Validation Loss: {val_loss:.4f}")

    # Print the current learning rate
    current_lr = scheduler.get_last_lr()[0]
    print(f"Epoch {epoch + 1}, Learning Rate: {current_lr:.6f}")



Epoch 0, Validation Loss: 0.9822
Epoch 1, Validation Loss: 0.0716
Epoch 1, Learning Rate: 0.004532
Epoch 2, Validation Loss: 0.0635
Epoch 2, Learning Rate: 0.003307
Epoch 3, Validation Loss: 0.0587
Epoch 3, Learning Rate: 0.001793
Epoch 4, Validation Loss: 0.0611
Epoch 4, Learning Rate: 0.000568
Epoch 5, Validation Loss: 0.0618
Epoch 5, Learning Rate: 0.000100


## Prediction - TO DO

In [9]:
a = np.ones((3, 2))
a = torch.tensor(a)
torch.flatten(a)

tensor([1., 1., 1., 1., 1., 1.], dtype=torch.float64)

In [144]:
def evaluate_model(model, initial_data, num_predictions=24):
    """
    Evaluate the LSTM to predict the next 24 hours recursively.

    Args:
        model (LSTMModel): Trained LSTM model.
        initial_data (torch.Tensor): Data from the last 7 days (shape: [168, num_features]).
        num_predictions (int): Number of hours to predict (default: 24).

    Returns:
        torch.Tensor: Predicted values for the next 24 hours (shape: [24, num_features]).
    """
    model.eval()
    predictions = []

    # Initialize hidden state with the last 7 days
    with torch.no_grad():
        input_seq = initial_data.unsqueeze(0)  # Shape: [1, seq_len=168, num_features]
        hidden = None  # Let the LSTM initialize hidden state

        # Process the last 7 days to initialize hidden state
        for t in range(initial_data.size(0)):
            _, hidden = model.lstm(input_seq[:, t:t+1, :], hidden)

        # Recursive prediction for the next 24 hours
        last_input = initial_data[-1, :].unsqueeze(0).unsqueeze(0)  # Shape: [1, 1, num_features]
        for _ in range(num_predictions):
            output, hidden = model.lstm(last_input, hidden)  # Predict next hour
            prediction = model.fc(output[:, -1, :])  # Map hidden state to output
            predictions.append(prediction.squeeze(0))

            # Use the predicted value as the next input
            last_input = prediction.unsqueeze(0).unsqueeze(0)

    return torch.stack(predictions)  # Shape: [24, num_features]

# Example usage
last_week_data = torch.tensor(data[-168:], dtype=torch.float32)  # Last 7 days of data
predictions = evaluate_model(model, last_week_data)
print(predictions)


C:\Users\Max Tost\AppData\Local\Temp\ipykernel_5640\4141761729.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  last_week_data = torch.tensor(data[-168:], dtype=torch.float32)  # Last 7 days of data


ValueError: LSTM: Expected input to be 2D or 3D, got 4D instead

## Uncertainty - TO DO

In [13]:
def monte_carlo_predictions(model, x, n_simulations):
    """
    Perform Monte Carlo Dropout predictions to estimate both 
    the mean prediction and uncertainty.

    Args:
        model (torch.nn.Module): The trained PyTorch model with dropout layers.
        x (torch.Tensor): Input tensor of shape (batch_size, seq_length, input_features).
        n_simulations (int): Number of stochastic forward passes to perform.

    Returns:
        tuple:
            - mean_pred (torch.Tensor): The mean prediction across all simulations.
              Shape: (batch_size, output_features).
            - uncertainty (torch.Tensor): The standard deviation of predictions 
              (representing uncertainty) across simulations.
              Shape: (batch_size, output_features).
    """
    # Set the model to train mode to enable dropout during inference
    # Dropout layers behave stochastically in train mode, which is necessary for Monte Carlo sampling
    model.train()

    # Perform n_simulations stochastic forward passes
    # Each simulation generates slightly different predictions due to dropout
    preds = torch.stack([model(x) for _ in range(n_simulations)])  # Shape: (n_simulations, batch_size, output_features)

    # Compute the mean prediction across all simulations
    mean_pred = preds.mean(dim=0)  # Shape: (batch_size, output_features)

    # Compute the standard deviation across simulations to estimate uncertainty
    uncertainty = preds.std(dim=0)  # Shape: (batch_size, output_features)

    return mean_pred, uncertainty


In [ ]:
test_x = [x for x, y in val_loader] # Extracting x and y from validation sets

In [ ]:
uncert = monte_carlo_predictions(model, test_x[0], 100) #finding uncertainty with one prediction

In [24]:
uncert = []
for x_t in test_x:
    uncert.append(monte_carlo_predictions(model, x_t, 100))

KeyboardInterrupt: 

In [19]:
test_x[0].shape

torch.Size([1, 168, 8])

In [21]:
model(test_x[1])

tensor([[ 8.8918e+02,  6.4641e+00,  1.5547e+01,  1.0813e+01, -6.0888e-01,
          2.5608e-01,  2.8174e-01, -4.7669e-01]], grad_fn=<AddmmBackward0>)